In [2]:
!pip install orjson

  Using cached orjson-3.9.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
Using cached orjson-3.9.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import json
import os
import orjson
import glob
import sys
import time
from collections import defaultdict
from datetime import datetime,timedelta
import math
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
import importlib
import boto3
import re

# Specify the path to your module
#module_path = '/root/DataScience-Repo/SageMaker/read_data_in_sagemaker.py'
module_path = '/root/Unified_LN_V2/DataScience-Repo/SageMaker/read_data_in_sagemaker.py'

# Create a spec for the module
module_spec = importlib.util.spec_from_file_location('read_data_in_sagemaker', module_path)

# Import the module
read_data_in_sagemaker = importlib.util.module_from_spec(module_spec)
module_spec.loader.exec_module(read_data_in_sagemaker)

s3 = boto3.client(service_name="s3",region_name = "us-east-2")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
file_list= read_data_in_sagemaker.list_objects_in_folder(s3, 'scien-ds-sm-root-backup', 'home/sushil/LN_Unified_V2/Client_Data_Auto/')
file_list

['home/sushil/LN_Unified_V2/Client_Data_Auto/040324_Hawaii_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Arkansas_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Bethpage_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_CPM_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_GESA_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Guardian_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_NCU_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Numerica_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/050324_SECU_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/130324_HFCU_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/180324_AAC_With_Bad_And_LN_Data.csv',
 'home/sushil/LN_Unified_V2/Client_Data_Auto/210324_LN_V2_Aut

In [6]:
df_all= pd.DataFrame()

for file in file_list:
    print(file)
    try:
        if file!= 'home/sushil/LN_Unified_V2/Client_Data_Auto/210324_LN_V2_Auto_Dataset_With_Performance_Race_Age_and_Gender.csv':
            temp= pd.read_csv(f's3://scien-ds-sm-root-backup/{file}')
            print(f"Client - {temp['client'].unique()}, Data Size - {temp.shape}")

            temp.columns= [i.lower() for i in temp.columns]

            df_all= pd.concat([df_all, temp]).reset_index(drop= True)
            print(f'Processed Data Size - {df_all.shape}')
        else:
            continue
    except Exception as e:
        print(e)

home/sushil/LN_Unified_V2/Client_Data_Auto/040324_Hawaii_With_Bad_And_LN_Data.csv
Client - ['Hawaii'], Data Size - (9923, 265)
Processed Data Size - (9923, 265)
home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Arkansas_With_Bad_And_LN_Data.csv
Client - ['Arkansas'], Data Size - (194180, 265)
Processed Data Size - (204103, 267)
home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Bethpage_With_Bad_And_LN_Data.csv
Client - ['Bethpage'], Data Size - (44372, 277)
Processed Data Size - (248475, 281)
home/sushil/LN_Unified_V2/Client_Data_Auto/050324_CPM_With_Bad_And_LN_Data.csv
Client - ['CPM'], Data Size - (19018, 265)
Processed Data Size - (267493, 281)
home/sushil/LN_Unified_V2/Client_Data_Auto/050324_GESA_With_Bad_And_LN_Data.csv
Client - ['GESA'], Data Size - (96823, 277)
Processed Data Size - (364316, 281)
home/sushil/LN_Unified_V2/Client_Data_Auto/050324_Guardian_With_Bad_And_LN_Data.csv
Client - ['Guardian'], Data Size - (112269, 265)
Processed Data Size - (476585, 281)
home/sushil/LN_

In [7]:
df_all.head(2)

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,channel,fico_cut,applicant_name,applicant_city,applicant_state,applicant_zip,applicant_address,applicant_dob,coapplicant_name,coapplicant_city,coapplicant_state,coapplicant_zip,coapplicant_address,coapplicant_dob,applicant_age,coapplicant_age,co_ever,co_12m,co_18m,auto_index,auto_score_name,auto_score,auto_reason1,auto_reason2,auto_reason3,auto_reason4,auto_reason5,bankcard_index,bankcard_score_name,bankcard_score,bankcard_reason1,bankcard_reason2,bankcard_reason3,bankcard_reason4,bankcard_reason5,short_term_lending_index,short_term_lending_score_name,short_term_lending_score,short_term_lending_reason1,short_term_lending_reason2,short_term_lending_reason3,short_term_lending_reason4,short_term_lending_reason5,telecommunications_index,telecommunications_score_name,telecommunications_score,telecommunications_reason1,telecommunications_reason2,telecommunications_reason3,telecommunications_reason4,telecommunications_reason5,crossindustry_index,crossindustry_score_name,crossindustry_score,crossindustry_reason1,crossindustry_reason2,crossindustry_reason3,crossindustry_reason4,crossindustry_reason5,custom_index,custom_score_name,custom_score,custom_reason1,custom_reason2,custom_reason3,custom_reason4,custom_reason5,inputprovidedfirstname,inputprovidedlastname,inputprovidedstreetaddress,inputprovidedcity,inputprovidedstate,inputprovidedzipcode,inputprovidedssn,inputprovideddateofbirth,inputprovidedphone,inputprovidedlexid,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,subjectactivityindex03month,subjectactivityindex06month,subjectactivityindex12month,subjectage,subjectdeceased,subjectssncount,subjectstabilityindex,subjectstabilityprimaryfactor,subjectabilityindex,subjectabilityprimaryfactor,subjectwillingnessindex,subjectwillingnessprimaryfactor,confirmationsubjectfound,confirmationinputname,confirmationinputdob,confirmationinputssn,confirmationinputaddress,sourcenonderogprofileindex,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationindex,businessassociationtimeoldest,businesstitleleadership,assetindex,assetindexprimaryfactor,assetownership,assetprop,assetpropindex,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,purchaseactivityindex,purchaseactivitycount,purchaseactivitydollartotal,derogseverityindex,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentseverityindex,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,ssnsubjectcount,ssndeceased,ssndatelowissued,ssnproblems,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofr

### Filtering data from invalid auto score

In [8]:
df_invalid= df_all[(df_all['auto_score'].isnull()) | (df_all['auto_score']<= 500)].reset_index(drop= True)
df_invalid.shape

(36884, 281)

In [10]:
req_cols= ['assetpropevercount', 'sourcenonderogcount06month', 'addronfilecount', 'addrinputcountyratio', 'assetpropnewestsaleprice', 'addrcurrenttaxvalue', 'addrinputavmvalue', 'addrinputavmvalue60month', 'addrinputlastsaleprice', 'addrinputtaxmarketvalue', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'assetpropcurrenttaxtotal', 'addrcurrentavmvalue', 'assetproppurchasetimeoldest', 'addrpreviouslengthofres', 'addrinputlengthofres', 'addrcurrenttaxmarketvalue', 'addrprevioustimenewest', 'addrcurrenttractratio', 'sourcenonderogcount12month', 'assetpropcurrentcount', 'addrinputsubjectcount', 'sourcenonderogcount', 'addrinputtaxvalue', 'assetpropeversoldcount', 'businessassociationtimeoldest', 'addrinputtractratio', 'derogtimenewest', 'addrchangecount24month', 'addrinputtimelastsale', 'educationinstitutionprivate', 'addrinputblockratio', 'businesstitleleadership', 'addrcurrentlengthofres', 'addrcurrentavmratio12monthprior', 'addrcurrentblockratio', 'derogcount', 'profliccount', 'addrinputavmratio60monthprior', 'addrcurrentcountyratio', 'assetpropsaletimeoldest', 'subjectrecordtimeoldest', 'addrcurrenttimenewest', 'proflictypecategory', 'criminalnonfelonytimenewest', 'lienjudgmentdollartotal', 'addrcurrentlastsalesprice', 'addrinputtimenewest', 'evictiontimenewest', 'bankruptcycount', 'addrcurrenttimelastsale', 'addrinputtimeoldest', 'addrlastmovetaxratiodiff', 'criminalnonfelonycount', 'sourcenonderogcount03month', 'addrchangecount03month', 'addrchangecount06month', 'addrchangecount12month', 'addrchangecount60month', 'addrcurrentavmratio60monthprior', 'addrcurrentavmvalue12month', 'addrcurrentavmvalue60month', 'addrcurrentcorrectional', 'addrcurrentdeedmailing', 'addrcurrentphoneservice', 'addrcurrentsubjectowned', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'addrinputavmvalue12month', 'addrinputdeedmailing', 'addrinputphoneservice', 'addrinputproblems', 'addrinputsubjectowned', 'addrlastmoveecontrajectory', 'addronfilecollege', 'addronfilecorrectional', 'addronfilehighrisk', 'addrpreviouscorrectional', 'addrprevioussubjectowned', 'addrprevioustimeoldest', 'alertregulatorycondition', 'assetownership', 'assetpersonal', 'assetpersonalcount', 'assetprop', 'assetpropnewestmortgagetype', 'assetproppurchasecount12month', 'assetproppurchasetimenewest', 'assetpropsalepurchaseratio', 'assetpropsaletimenewest', 'assetpropsoldcount12month', 'bankruptcychapter', 'bankruptcycount24month', 'bankruptcydismissed24month', 'bankruptcystatus', 'bankruptcytimenewest', 'businessassociation', 'criminalfelonycount', 'criminalfelonycount12month', 'criminalfelonytimenewest', 'criminalnonfelonycount12month', 'derogcount12month', 'educationattendance', 'educationevidence', 'educationinstitutionrating', 'educationprogramattended', 'evictioncount', 'evictioncount12month', 'inquiryauto12month', 'inquirybanking12month', 'inquirycollections12month', 'inquirynonshortterm12month', 'inquiryshortterm12month', 'inquirytelcom12month', 'lienjudgmentcount', 'lienjudgmentcount12month', 'lienjudgmentcourtcount', 'lienjudgmentforeclosurecount', 'lienjudgmentothercount', 'lienjudgmentsmallclaimscount', 'lienjudgmenttaxcount', 'lienjudgmenttimenewest', 'shorttermloanrequest', 'shorttermloanrequest12month', 'shorttermloanrequest24month', 'sourcecredheadertimenewest', 'sourcevoterregistration', 'subjectnewestrecord12month', 'subjectrecordtimenewest']
len(req_cols)

130

In [22]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 4.0 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [27]:
df_cont_1, df_cat_1= Univariates(df_invalid[req_cols], special_value= -1)

In [28]:
df_cont_1.shape, df_cat_1.shape

((130, 26), (0, 6))

In [29]:
df_cont_1

,variable,#of missing,#of records,%missing,#[-1],%[-1],avg,median,standard_deviation,skew,kurtosis,min,p_5,p_10,p_20,p_30,p_40,p_50,p_60,p_70,p_80,p_90,p_95,p_99,max,flag_excl
0,assetpropevercount,16163,36884,0.438212,19243,0.521717,-0.870518,-1.0,0.573784,6.688783,64.390389,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,2.000,10.00,0
1,sourcenonderogcount06month,16163,36884,0.438212,19243,0.521717,-0.837749,-1.0,0.604301,3.741131,13.364585,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.000,4.00,0
2,addronfilecount,16163,36884,0.438212,19243,0.521717,-0.538874,-1.0,1.955205,5.190305,30.911628,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,9.000,24.00,0
3,addrinputcountyratio,16163,36884,0.438212,19247,0.521825,-0.889196,-1.0,0.464779,8.175033,161.683158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.130,15.51,0
4,assetpropnewestsaleprice,16163,36884,0.438212,20553,0.557233,1523.917572,-1.0,20957.645544,17.230211,345.409901,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,702995.00,1
5,addrcurrenttaxvalue,16163,36884,0.438212,19243,0.521717,2132.087110,-1.0,23523.091088,14.758730,258.377074,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,47500.000,601900.00,0
6,addrinputavmvalue,16163,36884,0.438212,19894,0.539367,9566.693982,-1.0,106202.064341,57.575036,4505.463280,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,281012.000,8742023.00,0
7,addrinputavmvalue60month,16163,36884,0.438212,20023,0.542864,5826.098644,-1.0,43904.114307,15.806282,511.051197,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,196234.000,2382228.00,0
8,addrinputlastsaleprice,16163,36884,0.438212,20461,0.554739,2357.400898,-1.0,28236.762928,18.264385,466.736395,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,52426.400,1250000.00,0
9,addrinputtaxmarketvalue,16163,36884,0.438212,19247,0.521825,3933.116983,-1.0,35823.141643,12.199467,187.070580,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,162810.000,1089108.00,0


In [30]:
df_cont_2, df_cat_2= Univariates(df_invalid[df_invalid['auto_score']<= 100][req_cols], special_value= -1)

In [31]:
df_cont_2

,variable,#of missing,#of records,%missing,#[-1],%[-1],avg,median,standard_deviation,skew,kurtosis,min,p_5,p_10,p_20,p_30,p_40,p_50,p_60,p_70,p_80,p_90,p_95,p_99,max,flag_excl
0,assetpropevercount,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,sourcenonderogcount06month,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,addronfilecount,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,addrinputcountyratio,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,assetpropnewestsaleprice,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,addrcurrenttaxvalue,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,addrinputavmvalue,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,addrinputavmvalue60month,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,addrinputlastsaleprice,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,addrinputtaxmarketvalue,9897,9897,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [32]:
df_cont_3, df_cat_3= Univariates(df_invalid[df_invalid['auto_score']== 200][req_cols], special_value= -1)

In [33]:
df_cont_3

,variable,#of missing,#of records,%missing,#[-1],%[-1],avg,median,standard_deviation,skew,kurtosis,min,p_5,p_10,p_20,p_30,p_40,p_50,p_60,p_70,p_80,p_90,p_95,p_99,max,flag_excl
0,assetpropevercount,0,1478,0.0,0,0.000000,0.815291,0.000,1.247496,2.623993,10.888179,0.0,0.00,0.0,0.0,0.0,0.000,0.000,1.000,1.00,1.000,2.000,3.0000,5.0000,10.00,0
1,sourcenonderogcount06month,0,1478,0.0,0,0.000000,1.274696,1.000,0.560777,1.194084,1.805957,0.0,1.00,1.0,1.0,1.0,1.000,1.000,1.000,1.00,2.000,2.000,2.0000,3.0000,4.00,0
2,addronfilecount,0,1478,0.0,0,0.000000,5.464817,5.000,3.845675,1.101021,1.352789,0.0,1.00,1.0,2.0,3.0,4.000,5.000,6.000,7.00,8.600,11.000,13.0000,17.0000,24.00,0
3,addrinputcountyratio,0,1478,0.0,4,0.002706,0.553437,0.350,0.887615,7.326681,100.640177,-1.0,0.00,0.0,0.0,0.0,0.000,0.350,0.622,0.83,1.010,1.300,1.6415,3.0561,15.51,0
4,assetpropnewestsaleprice,0,1478,0.0,1310,0.886333,21377.766576,-1.000,75742.143278,4.321591,20.789735,-1.0,-1.00,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.00,-1.000,35975.000,179600.0000,393188.0000,702995.00,0
5,addrcurrenttaxvalue,0,1478,0.0,0,0.000000,29904.073072,0.000,83254.732565,3.636786,14.901518,0.0,0.00,0.0,0.0,0.0,0.000,0.000,0.000,0.00,8894.400,114820.000,217493.0000,403329.0000,601900.00,0
6,addrinputavmvalue,0,1478,0.0,651,0.440460,134134.444520,48112.500,376171.186607,17.167389,376.854949,-1.0,-1.00,-1.0,-1.0,-1.0,-1.000,48112.500,101000.000,154995.70,218339.400,336687.100,478382.8500,757050.4500,8742023.00,0
7,addrinputavmvalue60month,0,1478,0.0,780,0.527740,81692.715156,-1.000,144356.461619,4.522474,47.654495,-1.0,-1.00,-1.0,-1.0,-1.0,-1.000,-1.000,48332.200,95648.40,147520.200,247314.500,344582.0000,597880.5200,2382228.00,0
8,addrinputlastsaleprice,0,1478,0.0,1218,0.824087,33062.887009,-1.000,100842.199973,4.689372,31.362114,-1.0,-1.00,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.00,-1.000,128650.000,224035.0000,494230.0000,1250000.00,0
9,addrinputtaxmarketvalue,0,1478,0.0,4,0.002706,55153.829499,0.000,123189.586321,2.893061,10.236846,-1.0,0.00,0.0,0.0,0.0,0.000,0.000,0.000,0.00,90720.000,224242.300,325855.5000,556489.0000,1089108.00,0


In [34]:
df_cont_4, df_cat_4= Univariates(df_invalid[df_invalid['auto_score']== 222][req_cols], special_value= -1)

In [35]:
df_cont_4

,variable,#of missing,#of records,%missing,#[-1],%[-1],avg,median,standard_deviation,skew,kurtosis,min,p_5,p_10,p_20,p_30,p_40,p_50,p_60,p_70,p_80,p_90,p_95,p_99,max,flag_excl
0,assetpropevercount,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
1,sourcenonderogcount06month,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,addronfilecount,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
3,addrinputcountyratio,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
4,assetpropnewestsaleprice,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
5,addrcurrenttaxvalue,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
6,addrinputavmvalue,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
7,addrinputavmvalue60month,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
8,addrinputlastsaleprice,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
9,addrinputtaxmarketvalue,0,19243,0.0,19243,1.0,-1.000000,-1.0,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [37]:
df_cont_5, df_cat_5= Univariates(df_invalid[df_invalid['auto_score'].isnull()][req_cols], special_value= -1)

In [38]:
df_cont_5

,variable,#of missing,#of records,%missing,#[-1],%[-1],avg,median,standard_deviation,skew,kurtosis,min,p_5,p_10,p_20,p_30,p_40,p_50,p_60,p_70,p_80,p_90,p_95,p_99,max,flag_excl
0,assetpropevercount,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,sourcenonderogcount06month,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,addronfilecount,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,addrinputcountyratio,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,assetpropnewestsaleprice,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,addrcurrenttaxvalue,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,addrinputavmvalue,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,addrinputavmvalue60month,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,addrinputlastsaleprice,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,addrinputtaxmarketvalue,6266,6266,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [36]:
df_invalid['auto_score'].value_counts(dropna= False)

auto_score
222.0    19243
100.0     9897
NaN       6266
200.0     1478
Name: count, dtype: int64

In [39]:
# for var in sorted(req_cols):
#     print('*'*25)
#     print(var)
#     print({'#Total': len(df_invalid), '#Missing': df_invalid[var].isnull().sum()})
#     if len(df_invalid[var].unique())> 10:
#         display(pd.DataFrame(df_invalid[var].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])).T)
#     else:
#         display(pd.DataFrame(df_invalid[var].value_counts(dropna= False).sort_index()).T)